In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from pandas import Series,DataFrame
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as metrics

In [11]:
train_data = pd.read_table("A/a.txt",header=None)
test_data = pd.read_table("A/a-test.txt",header=None)

In [12]:
train_data.iloc[:,0] = train_data.iloc[:,0].str.replace('[','')
train_data.iloc[:,0] = train_data.iloc[:,0].str.replace(']','')
train_data = train_data.iloc[:,0].str.split('", ',expand=True)
train_data.columns=['addr','time','status','file','sent','request_time','uuid','referer','agent','forwarded','pno','Id','auth_ret']

test_data.iloc[:,0] = test_data.iloc[:,0].str.replace('[','')
test_data.iloc[:,0] = test_data.iloc[:,0].str.replace(']','')
test_data = test_data.iloc[:,0].str.split('", ',expand=True)
test_data.columns=['addr','time','status','file','sent','request_time','uuid','referer','agent','forwarded','pno','Id']

for i in train_data.columns:
    train_data[i] = train_data[i].str.replace('"','')
for i in test_data.columns:
    test_data[i] = test_data[i].str.replace('"','')
    
train_data.to_csv('A/a.csv',index=False)
test_data.to_csv('A/a-test.csv',index=False)


In [2]:
train_data = pd.read_csv('A/a.csv')
test_data = pd.read_csv('A/a-test.csv')

In [38]:
date = train_data['time'].str.split('T',expand=True)
date.columns=['date_','time_']
y = date['date_'].str.split('-',expand=True)
y.columns = ['year','month','day']
t = date['time_'].str.split('+',expand=True)
t.columns = ['time','more_time']
t1 = t['time'].str.split(':',expand=True)
t1.columns=['hour','minute','second']
t2 = t['more_time'].str.split(':',expand=True)
t2.columns=['minute','second']
#train_data['year'] = y['year']  只有2021
train_data['month'] = y['month']
train_data['day'] = y['day']
train_data['hour'] = t1['hour']
train_data['minute'] = t1['minute']
train_data['second'] = t1['second']
train_data['more_minute'] = t2['minute']
train_data['more_second'] = t2['second']

#train_data['year']=train_data['year'].astype(np.int32)
train_data['month']=train_data['month'].astype(np.int32)
train_data['day']=train_data['day'].astype(np.int32)
train_data['hour']=train_data['hour'].astype(np.int32)
train_data['minute']=train_data['minute'].astype(np.int32)
train_data['second']=train_data['second'].astype(np.int32)
train_data['more_minute']=train_data['more_minute'].astype(np.int32)
train_data['more_second']=train_data['more_second'].astype(np.int32)


date = test_data['time'].str.split('T',expand=True)
date.columns=['date_','time_']
y = date['date_'].str.split('-',expand=True)
y.columns = ['year','month','day']
t = date['time_'].str.split('+',expand=True)
t.columns = ['time','more_time']
t1 = t['time'].str.split(':',expand=True)
t1.columns=['hour','minute','second']
t2 = t['more_time'].str.split(':',expand=True)
t2.columns=['minute','second']
#test_data['year'] = y['year']
test_data['month'] = y['month']
test_data['day'] = y['day']
test_data['hour'] = t1['hour']
test_data['minute'] = t1['minute']
test_data['second'] = t1['second']
test_data['more_minute'] = t2['minute']
test_data['more_second'] = t2['second']

#test_data['year']=test_data['year'].astype(np.int32)
test_data['month']=test_data['month'].astype(np.int32)
test_data['day']=test_data['day'].astype(np.int32)
test_data['hour']=test_data['hour'].astype(np.int32)
test_data['minute']=test_data['minute'].astype(np.int32)
test_data['second']=test_data['second'].astype(np.int32)
test_data['more_minute']=test_data['more_minute'].astype(np.int32)
test_data['more_second']=test_data['more_second'].astype(np.int32)


In [39]:
lbe = preprocessing.LabelEncoder()

forwarded = train_data['forwarded'].str.split(',',expand=True)
forwarded.columns = ['client','proxy1','proxy2','proxy3']
forwarded['proxy1'].fillna('-')
forwarded['proxy2'].fillna('-')
forwarded['proxy3'].fillna('-')
train_data['client'] = lbe.fit_transform(list(forwarded['client'].values))
train_data['proxy1'] = lbe.fit_transform(list(forwarded['proxy1'].values))
train_data['proxy2'] = lbe.fit_transform(list(forwarded['proxy2'].values))
train_data['proxy3'] = lbe.fit_transform(list(forwarded['proxy3'].values))


forwarded = test_data['forwarded'].str.split(',',expand=True)
forwarded.columns = ['client','proxy1','proxy2','proxy3']
forwarded['proxy1'].fillna('-')
forwarded['proxy1'].fillna('-')
forwarded['proxy2'].fillna('-')
test_data['client'] = lbe.fit_transform(list(forwarded['client'].values))
test_data['proxy1'] = lbe.fit_transform(list(forwarded['proxy1'].values))
test_data['proxy2'] = lbe.fit_transform(list(forwarded['proxy2'].values))
test_data['proxy3'] = lbe.fit_transform(list(forwarded['proxy3'].values))

In [40]:
train_data.head()

,addr,time,status,file,sent,request_time,uuid,referer,agent,forwarded,...,day,hour,minute,second,more_minute,more_second,client,proxy1,proxy2,proxy3
0,6c455e37ea1f3f43443ecbe46bcdb4bb,2021-03-22T12:20:00+08:00,200,/c1/2021/02/20_0/481B858722405F7E39837605A91E8...,916,0.003,cc8af1795c1f41d5928a812894582677,-,ImgoTV-ipad-AppStore/6.3.8.20123119 CFNetwork/...,6c455e37ea1f3f43443ecbe46bcdb4bb,...,22,12,20,0,8,0,876039,224,23,2
1,64eaa6e58e5bd71d54dd7565481a2f1d,2021-03-22T12:20:00+08:00,200,/c1/2021/03/20_0/F585736A7DD210F89DE3AEA196944...,966,0.002,e08080787bfb45dcbb06b2a2794bbd19,-,Dalvik/2.1.0 (Linux; U; Android 9; ART-AL00x B...,-,...,22,12,20,0,8,0,0,224,23,2
2,4d4c7b25b890132ee432bebab3940966,2021-03-22T12:20:00+08:00,200,/c1/2020/06/09_1/B635E0C143F43265B4522F4F5C665...,908,0.000,16ef9189edb047da9eb2c80f7c699989,-,Dalvik/2.1.0 (Linux; U; Android 10; PCNM00 Bui...,-,...,22,12,20,0,8,0,0,224,23,2
3,da177836163927897f26c718d1daf85f,2021-03-22T12:20:00+08:00,200,/c1/2017/05/23_0/9682886A50AEC846687B165FA70C3...,897,0.001,2ae94ad529634ba092012ee451854ba4,https://www.mgtv.com/b/166040/3557859.html,Mozilla/5.0 (iPad; CPU OS 12_4_8 like Mac OS X...,aaa477e9b59a19d1a1a6f5fc0d869bfe,...,22,12,20,0,8,0,1379746,224,23,2
4,affef57aacffb8aa6e911659e55f54b2,2021-03-22T12:20:00+08:00,200,/c1/internettv/2020/09/11_0/1B19375BF694C900E2...,872,0.001,7e1e28044456459fab1d3aa865670398,-,Go-http-client/1.1,bb34f10e2b27ced77ad5384e7ab95032,...,22,12,20,0,8,0,1513715,224,23,2


In [41]:
#LabelEncoder
lbe = preprocessing.LabelEncoder()
train_data['addr'] = lbe.fit_transform(list(train_data['addr'].values))
train_data['uuid'] = lbe.fit_transform(list(train_data['uuid'].values))
train_data['Id'] = lbe.fit_transform(list(train_data['Id'].values))

lbe = preprocessing.LabelEncoder()
test_data['addr'] = lbe.fit_transform(list(test_data['addr'].values))
test_data['uuid'] = lbe.fit_transform(list(test_data['uuid'].values))
test_data['Id'] = lbe.fit_transform(list(test_data['Id'].values))

#删除forwarded
train_data = train_data.drop(['forwarded'],axis=1)
test_data = test_data.drop(['forwarded'],axis=1)
#删除file
train_data = train_data.drop(['file'],axis=1)
test_data = test_data.drop(['file'],axis=1)
#删除time
train_data = train_data.drop(['time'],axis=1)
test_data = test_data.drop(['time'],axis=1)
#request_time
train_data['request_time'] = train_data['request_time'].astype(np.float64)
test_data['request_time'] = test_data['request_time'].astype(np.float64)
#pno
train_data['pno'] = train_data['pno'].astype(np.int64)
test_data['pno'] = test_data['pno'].astype(np.int64)
#status
train_data = train_data.drop(['status'],axis=1)
test_data = test_data.drop(['status'],axis=1)

#sent
train_data['sent'] = train_data['sent'].astype(np.int64)
test_data['sent'] = test_data['sent'].astype(np.int64)

In [82]:
train_data['https']='0'
train_data['click']='0'
index = train_data['referer'].str.contains('https')  #https列，https为1，其余为0
train_data['https'][index] = '1'
train_data['https'].fillna('0')
index = train_data['referer'].str.contains('://')   #click列，可点击为1，其余为0
train_data['click'][index] = '1'
train_data['click'].fillna('0')
index = train_data['referer'].str.contains('-')   #referer列，-为0，其余为1
train_data['referer'][index] = '0'
train_data['referer'][~index] = '1'
train_data['referer'] = train_data['referer'].astype(np.int32)
train_data['https'] = train_data['https'].astype(np.int32)
train_data['click'] = train_data['click'].astype(np.int32)

test_data['https']='0'
test_data['click']='0'
index = test_data['referer'].str.contains('https')  #https列，https为1，其余为0
test_data['https'][index] = '1'
test_data['https'].fillna('0')
index = test_data['referer'].str.contains('://')   #click列，可点击为1，其余为0
test_data['click'][index] = '1'
test_data['click'].fillna('0')
index = test_data['referer'].str.contains('-')   #referer列，-为0，其余为1
test_data['referer'][index] = '0'
test_data['referer'][~index] = '1'
test_data['referer'] = test_data['referer'].astype(np.int32)
test_data['https'] = test_data['https'].astype(np.int32)
test_data['click'] = test_data['click'].astype(np.int32)

/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 

In [98]:
train_data.head()

,addr,sent,request_time,uuid,referer,agent,pno,Id,auth_ret,month,...,minute,second,more_minute,more_second,client,proxy1,proxy2,proxy3,https,click
0,1861642,916,0.003,11565578,0,ImgoTV-ipad-AppStore/6.3.8.20123119 CFNetwork/...,1101021,0,0,3,...,20,0,8,0,876039,224,23,2,0,0
1,1734847,966,0.002,12694739,0,Dalvik/2.1.0 (Linux; U; Android 9; ART-AL00x B...,1101010,0,1,3,...,20,0,8,0,0,224,23,2,0,0
2,1329581,908,0.000,1299164,0,Dalvik/2.1.0 (Linux; U; Android 10; PCNM00 Bui...,1131010,0,0,3,...,20,0,8,0,0,224,23,2,0,0
3,3753014,897,0.001,2428906,1,Mozilla/5.0 (iPad; CPU OS 12_4_8 like Mac OS X...,1121,0,0,3,...,20,0,8,0,1379746,224,23,2,1,1
4,3027860,872,0.001,7130623,0,Go-http-client/1.1,1141,0,0,3,...,20,0,8,0,1513715,224,23,2,0,0


In [97]:
test_data.head()

,addr,sent,request_time,uuid,referer,agent,pno,Id,month,day,...,minute,second,more_minute,more_second,client,proxy1,proxy2,proxy3,https,click
0,192224,852,0.001,825906,0,Go-http-client/1.1,1141,0,4,17,...,20,20,8,0,470520,137,11,2,0,0
1,448007,869,0.000,1285467,0,Go-http-client/1.1,1101141,0,4,19,...,24,48,8,0,262788,150,11,2,0,0
2,935758,832,0.001,342733,0,Go-http-client/1.1,1141,0,4,17,...,23,12,8,0,262788,9,11,2,0,0
3,453206,981,0.001,155944,0,Dalvik/2.1.0 (Linux; U; Android 10; RMX2121 Bu...,1010,0,4,18,...,21,55,8,0,0,195,11,2,0,0
4,1324854,827,0.001,821744,0,Go-http-client/1.1,1141,0,4,18,...,21,33,8,0,277176,10,11,2,0,0


In [9]:
#index = train_data['referer'].str.contains('-')
#train_data['referer'][index] = '0'
#index = train_data['referer'].str.contains('://')
#train_data['referer'][index] = '2'
#index = train_data['referer'].str.contains('http%')
#train_data['referer'][index] = '3'
# index = train_data['referer'].str.contains('https%')
# train_data['referer'][index] = '3'
# train_data['referer'] = train_data['referer'].astype(np.int32)

# index = test_data['referer'].str.contains('-')
# test_data['referer'][index] = '1'
# index = test_data['referer'].str.contains('://')
# test_data['referer'][index] = '2'
# index = test_data['referer'].str.contains('http%')
# test_data['referer'][index] = '3'
# index = test_data['referer'].str.contains('https%')
# test_data['referer'][index] = '3'
# test_data['referer'] = test_data['referer'].astype(np.int32)

/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/wyq/an

In [99]:
train_data.to_csv('A/train.csv',index=False)
test_data.to_csv('A/test.csv',index=False)

In [11]:
test_data.head()

,addr,status,sent,request_time,uuid,referer,agent,forwarded,pno,Id,month,day,hour,minute,second,more_minute,more_second
0,192224,200,852,0.001,825906,1,Go-http-client/1.1,470897,1141,0,4,17,21,20,20,8,0
1,448007,200,869,0.000,1285467,1,Go-http-client/1.1,263006,1101141,0,4,19,20,24,48,8,0
2,935758,200,832,0.001,342733,1,Go-http-client/1.1,262992,1141,0,4,17,21,23,12,8,0
3,453206,200,981,0.001,155944,1,Dalvik/2.1.0 (Linux; U; Android 10; RMX2121 Bu...,0,1010,0,4,18,12,21,55,8,0
4,1324854,200,827,0.001,821744,1,Go-http-client/1.1,277398,1141,0,4,18,12,21,33,8,0


In [122]:
train_data.head(50)

,addr,sent,request_time,uuid,referer,agent,pno,Id,auth_ret,month,...,minute,second,more_minute,more_second,client,proxy1,proxy2,proxy3,https,click
0,1861642,916,0.003,11565578,0,ImgoTV-ipad-AppStore/6.3.8.20123119 CFNetwork/...,1101021,0,0,3,...,20,0,8,0,876039,224,23,2,0,0
1,1734847,966,0.002,12694739,0,Dalvik/2.1.0 (Linux; U; Android 9; ART-AL00x B...,1101010,0,1,3,...,20,0,8,0,0,224,23,2,0,0
2,1329581,908,0.000,1299164,0,Dalvik/2.1.0 (Linux; U; Android 10; PCNM00 Bui...,1131010,0,0,3,...,20,0,8,0,0,224,23,2,0,0
3,3753014,897,0.001,2428906,1,Mozilla/5.0 (iPad; CPU OS 12_4_8 like Mac OS X...,1121,0,0,3,...,20,0,8,0,1379746,224,23,2,1,1
4,3027860,872,0.001,7130623,0,Go-http-client/1.1,1141,0,0,3,...,20,0,8,0,1513715,224,23,2,0,0
5,1392222,887,0.000,4937328,0,Dalvik/2.1.0 (Linux; U; Android 9; S1W Build/P...,1010,0,0,3,...,20,0,8,0,655039,224,23,2,0,0
6,1622712,944,0.001,5509461,0,Dalvik/2.1.0 (Linux; U; Android 9; vivo X21A B...,1010,0,0,3,...,20,0,8,0,0,224,23,2,0,0
7,2639044,968,0.000,1815309,1,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,1030,0,0,3,...,20,0,8,0,1417548,224,23,2,1,1
8,3205499,838,0.001,6158338,0,ImgoTV-ipad-AppStore/6.3.9.21030517 CFNetwork/...,1101021,0,0,3,...,20,0,8,0,400640,224,23,2,0,0
9,2658339,974,0.001,13261125,0,Dalvik/2.1.0 (Linux; U; Android 9; V1945A Buil...,1010,0,0,3,...,20,0,8,0,1249957,224,23,2,0,0


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3159000 entries, 0 to 3158999
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   addr          int64  
 1   status        int32  
 2   sent          int64  
 3   request_time  float64
 4   uuid          int64  
 5   referer       int32  
 6   agent         object 
 7   forwarded     int64  
 8   pno           int64  
 9   Id            int64  
 10  month         int32  
 11  day           int32  
 12  hour          int32  
 13  minute        int32  
 14  second        int32  
 15  more_minute   int32  
 16  more_second   int32  
dtypes: float64(1), int32(9), int64(6), object(1)
memory usage: 301.3+ MB


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14477290 entries, 0 to 14477289
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   addr          int64  
 1   status        int32  
 2   sent          int64  
 3   request_time  float64
 4   uuid          int64  
 5   referer       int32  
 6   agent         object 
 7   forwarded     int64  
 8   pno           int64  
 9   Id            int64  
 10  auth_ret      int64  
 11  month         int32  
 12  day           int32  
 13  hour          int32  
 14  minute        int32  
 15  second        int32  
 16  more_minute   int32  
 17  more_second   int32  
dtypes: float64(1), int32(9), int64(7), object(1)
memory usage: 1.5+ GB


In [147]:
train_data = pd.read_csv('A/train.csv')
test_data = pd.read_csv('A/test.csv')

In [148]:
train_data['agent'].fillna('0',inplace=True)

In [141]:
index = train_data['agent'].str.contains('-|\?')
train_data['agent'][index] = '0'

/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [133]:
index = train_data['agent'].str.contains('Dalvik|Imgo|Mozilla|Go|missing|AppStore|appstore|okhttp|CFNetwork|python-requests|Win64')
len(train_data['agent'][index])

14477288

In [155]:
index = train_data['agent'].str.contains('AppStore|appstore|Imgo|imgo')

In [156]:
train_data['agent'][index]

0           ImgoTV-ipad-AppStore/6.3.8.20123119 CFNetwork/...
1           Dalvik/2.1.0 (Linux; U; Android 9; ART-AL00x B...
2           Dalvik/2.1.0 (Linux; U; Android 10; PCNM00 Bui...
5           Dalvik/2.1.0 (Linux; U; Android 9; S1W Build/P...
6           Dalvik/2.1.0 (Linux; U; Android 9; vivo X21A B...
                                  ...                        
14477282    MGTV-iPhone-appstore/6.1.3.1812061610 CFNetwor...
14477283    Dalvik/2.1.0 (Linux; U; Android 10; PCHM10 Bui...
14477285    Dalvik/2.1.0 (Linux; U; Android 10; TAS-AN00 B...
14477288    Dalvik/2.1.0 (Linux; U; Android 10; JEF-AN20 B...
14477289    Dalvik/2.1.0 (Linux; U; Android 10; WKG-AN00 B...
Name: agent, Length: 6088132, dtype: object

In [136]:
index = train_data['agent'].str.contains('AppStore|appstore')

981090     ???
8650801    ???
8699905    ???
8903881    ???
Name: agent, dtype: object

In [134]:
list(train_data['agent'][~index])

['NBMDULFGRMOODQIVZHBXXVARUPOXASE', 'BPENFVNIRFVTGNMEOJGVKJWVWKBQREL']

In [45]:
list(train_data['agent'].unique())

['ImgoTV-ipad-AppStore/6.3.8.20123119 CFNetwork/1128.0.1 Darwin/19.6.0',
 'Dalvik/2.1.0 (Linux; U; Android 9; ART-AL00x Build/HUAWEIART-AL00x) imgotv-aphone-6.8.5',
 'Dalvik/2.1.0 (Linux; U; Android 10; PCNM00 Build/QKQ1.190915.002) imgotv-aphone-6.7.4',
 'Mozilla/5.0 (iPad; CPU OS 12_4_8 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 VideoArticle/9.4.9 JsSdk/2.0 NetType/WIFI (Video 5.4.9 12.4.8) newiPad',
 'Go-http-client/1.1',
 'Dalvik/2.1.0 (Linux; U; Android 9; S1W Build/PPR1.180610.011) imgotv-aphone-6.8.1',
 'Dalvik/2.1.0 (Linux; U; Android 9; vivo X21A Build/PKQ1.180819.001) imgotv-aphone-6.8.5',
 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
 'ImgoTV-ipad-AppStore/6.3.9.21030517 CFNetwork/1209 Darwin/20.2.0',
 'Dalvik/2.1.0 (Linux; U; Android 9; V1945A Build/PKQ1.190626.001) imgotv-aphone-6.8.4',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:51.0) Gecko/20100101 Firefox/51.0.1',
 'Dal

In [100]:
#agent目前没有想法，先删了跑跑
train_data = train_data.drop(['agent'],axis=1)
test_data = test_data.drop(['agent'],axis=1)

In [101]:
X_train = train_data.drop(['auth_ret'],axis=1)
Y_train = train_data['auth_ret']

In [102]:
X_train.head()

,addr,sent,request_time,uuid,referer,pno,Id,month,day,hour,minute,second,more_minute,more_second,client,proxy1,proxy2,proxy3,https,click
0,1861642,916,0.003,11565578,0,1101021,0,3,22,12,20,0,8,0,876039,224,23,2,0,0
1,1734847,966,0.002,12694739,0,1101010,0,3,22,12,20,0,8,0,0,224,23,2,0,0
2,1329581,908,0.000,1299164,0,1131010,0,3,22,12,20,0,8,0,0,224,23,2,0,0
3,3753014,897,0.001,2428906,1,1121,0,3,22,12,20,0,8,0,1379746,224,23,2,1,1
4,3027860,872,0.001,7130623,0,1141,0,3,22,12,20,0,8,0,1513715,224,23,2,0,0


In [6]:
#报错：y_pred和pred是Dataset，不能做基本的操作
def custom_f1_eval(y_true, y_pred):
    num_TP = 0
    threshold = 0.5
    for i in range(len(y_true)):
        if y_pred >= threshold:
            num_pred += 1
            if y_true == 1:
                num_TP += 1
    precision = num_TP / num_pred
    recall = num_TP / len(y_true[y_true == 1])
    f1 = 2 * precision * recall / (precision + recall)
    return "f1", f1_mean, True

def f1_loss(y, pred):
    beta = 2
    p = 1. / (1 + np.exp(-pred))
    grad = p * ((beta - 1) * y + 1) - beta * y
    hess = ((beta - 1) * y + 1) * p * (1.0 - p)

    return grad, hess

In [103]:
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.where(y_hat < 0.5, 0, 1)  
    return 'f1', metrics.f1_score(y_true, y_hat, average='macro'), True

In [105]:
feature_name = X_train.columns.tolist()
category_feature = [
    'addr',
#    'file',
    'uuid',
    'referer',
#    'agent', #UA
    'client',
    'proxy1',
    'proxy2',
    'proxy3',
    'pno',
    'Id',
    'https',
    'click'
]
params = {
    'objective': 'binary',
    'metric':'auc',
    'num_leaves': 2**4-1,
    'learning_rate': 0.1,
    'feature_fraction':0.75,
    'bagging_fraction':0.75,
    'bagging_freq':5,
    'seed':1,
    'verbose':1
}


NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=True,random_state=2)
kf = kfold.split(X_train,Y_train)
cv_pred = np.zeros(test_data.shape[0])
valid = 0

for i,(train_fold,validate) in enumerate(kf):
    if i==1:
        break
    x_train,x_validate = X_train.iloc[train_fold],X_train.iloc[validate]
    y_train,y_validate = Y_train.iloc[train_fold],Y_train.iloc[validate]
    lgb_train = lgb.Dataset(x_train,y_train,categorical_feature=category_feature)
    lgb_eval = lgb.Dataset(x_validate,y_validate,reference=lgb_train,categorical_feature=category_feature)
    eval_result={}
    gbm = lgb.train(
        params,
        lgb_train,
        num_boost_round = 1000,
        valid_sets= (lgb_train,lgb_eval),
        feature_name = feature_name,
        categorical_feature = category_feature,
        verbose_eval=20, 
        evals_result = eval_result,
        early_stopping_rounds = 50,
        #fobj=f1_loss,
        #feval=custom_f1_eval
        feval=lgb_f1_score
    )
    cv_pred += gbm.predict(test_data,num_iteration=gbm.best_iteration)

#cv_pred /= NFOLDS


df = pd.DataFrame(np.around(cv_pred).astype(np.int32))
df.to_csv('A/submission.csv',index=False,header=False)   

[LightGBM] [Info] Number of positive: 2156760, number of negative: 9425072
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.229811 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1965
[LightGBM] [Info] Number of data points in the train set: 11581832, number of used features: 19


/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/wyq/anaconda3/envs/LGBM/lib/python3.6/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.186219 -> initscore=-1.474766
[LightGBM] [Info] Start training from score -1.474766
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.714673	training's f1: 0.597502	valid_1's auc: 0.7147	valid_1's f1: 0.597217
[40]	training's auc: 0.717187	training's f1: 0.606371	valid_1's auc: 0.717074	valid_1's f1: 0.606019
[60]	training's auc: 0.718442	training's f1: 0.609557	valid_1's auc: 0.718396	valid_1's f1: 0.609336
[80]	training's auc: 0.718887	training's f1: 0.62233	valid_1's auc: 0.718752	valid_1's f1: 0.622038
[100]	training's auc: 0.719433	training's f1: 0.623888	valid_1's auc: 0.719266	valid_1's f1: 0.623525
[120]	training's auc: 0.719837	training's f1: 0.624846	valid_1's auc: 0.71961	valid_1's f1: 0.62454
[140]	training's auc: 0.720118	training's f1: 0.625259	valid_1's auc: 0.719847	valid_1's f1: 0.624879
[160]	training's auc: 0.720368	training's f1: 0.625994	valid_1's auc: 0.720051	valid_1's f1: 0.62563

In [10]:
cv_pred /= NFOLDS
df = pd.DataFrame(np.around(cv_pred).astype(np.int32))
df.to_csv('A/submission.csv',index=False,header=False)   